In [ ]:
import math

import efd_utils
import df_utils

In [ ]:
location = "tucson"

In [ ]:
client = efd_utils.get_client(location)

In [ ]:
# Only Camera systems follow this pattern
cameras = ['ATCamera', 'CCCamera', 'MTCamera']
cscs = [efd_utils.CSC.from_entry(camera) for camera in cameras]

In [ ]:
summary_state = 4  # OFFLINE
time_window = 10.0  # seconds
time_format = '%Y-%m-%dT%H:%M:%S.%f'

In [ ]:
print("#########################################################")
print("#               OFFLINE State Transition                #")
print("#########################################################")
for csc in cscs:
    query = efd_utils.get_base_query(columns=["private_sndStamp",
                                              "summaryState"],
                                     csc_name=csc.name,
                                     csc_index=csc.index,
                                     topic_name="logevent_summaryState")

    query += f" WHERE summaryState={summary_state}"
    query += " " + efd_utils.get_time_clause(last=True)

    ss_df = await client.query(query)
    try:
        ss_df = df_utils.convert_timestamps(ss_df, ["private_sndStamp"])
    except KeyError:
        pass

    query = efd_utils.get_base_query(columns=["private_sndStamp",
                                              "substate"],
                                     csc_name=csc.name,
                                     csc_index=csc.index,
                                     topic_name="logevent_offlineDetailedState")

    query += efd_utils.get_time_clause(last=True, limit=2)

    ods_df = await client.query(query)
    try:
        ods_df = df_utils.convert_timestamps(ods_df, ["private_sndStamp"])
    except KeyError:
        pass

    print("---------------------------------------------------------")
    print(f"CSC: {csc.full_name}")
    try:
        print(f"Time of Summary State: {ss_df.private_sndStamp[0].strftime(time_format)}")
    except AttributeError:
        print(f"summaryState event not present for {csc.full_name}")
    try:
        delta = df_utils.time_delta(ss_df.private_sndStamp.values[0],
                                    ods_df.private_sndStamp.values[0])
        if math.fabs(delta) > time_window:
            print(f"Large delay in offlineDetailedState publish: {delta:.1f} seconds")
    except AttributeError:
        pass
    try:
        print(f"First Offline Detailed State: {ods_df.substate.values[1]}")
        print(f"Second Offline Detailed State: {ods_df.substate.values[0]}")
    except AttributeError:
        print(f"offlineDetailedState event not present for {csc.full_name}")